# Shallow Circuits and the Variational Principle

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

# Quantum Approximate Optimization Algorithm

Approximating the adiabatic pathway

Resolution

Overlap with the ground state

## Code

In [2]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.wrapper import execute as q_execute
from qiskit.tools.qi.pauli import Pauli
from qiskit.tools.apps.optimization import eval_hamiltonian 
from qiskit_aqua.operator import Operator
from qiskit_aqua import get_initial_state_instance
from qiskit import Aer
from qiskit.tools.qi.qi import state_fidelity

from scipy.optimize import minimize
from functools import reduce
import itertools

np.set_printoptions(precision=2)

### Parameters

#### Defintion of the problem

As a toy example, we will try to minimize the simple ising problem defined by the hamiltonian $H=-\sigma^Z_1 \otimes \sigma^Z_2$.

In [3]:
n_qubits = 2 # size of the system (= number of nodes in the graph)
J = np.array([[0,1],[0,0]]) # coefficients J_{i,j} in the Ising Model (= adjacency matrix of the graph)

#### Optimization parameters

In [4]:
n_iter = 10 # number of iterations of the optimization procedure
n_beta_gamma = 2 # number of times we apply U(beta)U(gamma) to the initial state
beta = np.random.uniform(0, np.pi*2, n_beta_gamma) # initial beta values
gamma = np.random.uniform(0, np.pi*2, n_beta_gamma) # initial gamma values

### Preparing the hamiltonians

The three functions below return Operators, a class in Aqua that allows among other things "evaluation" (computation of the mean value of the operator) and "evolution" (exponentiation of the operator)

In [5]:
def pauli_z(qubit, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[qubit], np.zeros(n_qubits))]])

def pauli_x(qubit, coeff):
    eye = np.eye((n_qubits))
    return Operator([[1, Pauli(np.zeros(n_qubits), eye[qubit])]])

def product_pauli_z(q1, q2, coeff):
    eye = np.eye((n_qubits))
    return Operator([[coeff, Pauli(eye[q1], np.zeros(n_qubits)) * Pauli(eye[q2], np.zeros(n_qubits))]])

#### Mixer Hamiltonian
Reference hamiltonian defined by $H_m = \sum_i^n \sigma^X_i$

In Aqua, Operators can be added with as simple `+` between them. However, the Python function `sum` doesn't work and need to be rewritten using `reduce`

In [6]:
Hm = reduce(lambda x,y:x+y,
            [pauli_x(i, 1) 
             for i in range(n_qubits)])
Hm.to_matrix()

#### Cost Hamiltonian
Cost hamiltonian defined by $H_c=-\sum_i^n J_{ij} \sigma^Z_i \otimes \sigma^Z_j$

In [7]:
Hc = reduce(lambda x,y:x+y,
            [product_pauli_z(i,j, -J[i,j]) # J[i,j] as a first parameter
             for i,j in itertools.product(range(n_qubits), repeat=2)])
Hc.to_matrix()

### Preparing the initial state
The initial state is a uniform superposition of all the states $|q_1,...,q_n\rangle$

In [8]:
init_state_vect = [1 for i in range(2**n_qubits)]
init_state = get_initial_state_instance('CUSTOM')
init_state.init_args(n_qubits, state_vector=init_state_vect)

### Constructing the initial circuit

The initial circuit prepares the initial state

In [9]:
qr = QuantumRegister(n_qubits)
circuit_init = init_state.construct_circuit('circuit', qr)

### Adding the unitary matrices to the circuit
The total circuit builds the state $U(H_m,\beta_k)U(H_c,\gamma_k)...U(H_m,\beta_1)U(H_c,\gamma_1) | \psi_0 \rangle$

The `evolve` takes a hamiltonian $H$ and an angle $t$ and returns a circuit component made of the unitary matrix $e^{j H t}$

In [10]:
def evolve(hamiltonian, angle, quantum_registers):
    return hamiltonian.evolve(None, angle, 'circuit', 1,
                              quantum_registers=quantum_registers,
                              expansion_mode='suzuki',
                              expansion_order=3)

To create the circuit, we need to compose the different unitary matrice given by `evolve`. In Qiskit, composition of operators works with a `+`, but for the same reason as for Operators, we need to recreate the function `sum` manually using `reduce` (as in the [official implementation](https://github.com/Qiskit/aqua/blob/master/qiskit_aqua/algorithms/adaptive/qaoa/varform.py) of QAOA)

In [11]:
def create_circuit(gamma, beta):
    circuit_evolv = reduce(lambda x,y: x+y, [evolve(Hm, beta[i], qr) + evolve(Hc, gamma[i], qr)
                                             for i in range(n_beta_gamma)])
    circuit = circuit_init + circuit_evolv
    return circuit

We now create a function `evaluate_circuit` that takes a single vector `gamma_beta` (the concatenation of `gamma` and `beta`) and returns $\langle H_c \rangle = \langle \psi | H_c | \psi \rangle$ where $\psi$ is defined by the circuit created with the function above.

In [12]:
def evaluate_circuit(gamma_beta):
    n = len(gamma_beta)//2
    circuit = create_circuit(gamma_beta[:n], gamma_beta[n:])
    return np.real(Hc.eval("matrix", circuit, 'statevector_simulator')[0]) # the value should always be real in theory, but for numerical reasons the imaginary part can be a very small number

In [13]:
evaluate_circuit(np.concatenate([gamma, beta]))

0.07539864506320798

### Optimization procedure

In [14]:
result = minimize(evaluate_circuit, np.concatenate([gamma, beta]), method='L-BFGS-B')

In [15]:
result

      fun: -0.9999999999998554
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([7.22e-07, 0.00e+00, 0.00e+00, 1.68e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 90
      nit: 9
   status: 0
  success: True
        x: array([3.93, 1.2 , 2.7 , 1.18])

### Analysis of the results

We create a circuit using the optimal parameters found.

In [16]:
circuit = create_circuit(result['x'][:n_beta_gamma], result['x'][n_beta_gamma:])

We use the `statevector_simulator` backend in order to display the state created by the circuit.

In [17]:
backend = Aer.get_backend('statevector_simulator')
job = q_execute(circuit, backend)
state = np.asarray(job.result().get_statevector(circuit))
print(np.absolute(state))
print(np.angle(state))

[7.07e-01 1.89e-07 1.89e-07 7.07e-01]
[-0.79  0.83  0.83 -0.79]


We see that the state is approximately $e^{0.79j} \frac{1}{\sqrt{2}} \left( |00 \rangle + |11 \rangle \right)$. It corresponds to a uniform superposition of the two solutions of the classicial problem: $(\sigma_1=1$, $\sigma_2=1)$ and $(\sigma_1=-1$, $\sigma_2=-1)$

Let's now try to evaluate the operators $\sigma^Z_1$ and $\sigma^Z_2$ independently:

In [18]:
Z0 = pauli_z(0, 1)
Z1 = pauli_z(1, 1)

In [19]:
print(Z0.eval("matrix", circuit, "statevector_simulator")[0])
print(Z1.eval("matrix", circuit, "statevector_simulator")[0])

(1.6653345369377348e-16+0j)
(1.6653345369377348e-16+0j)


We see that both are approximatively equal to zero. It's expected given the state we found above and corresponds a typical quantum behavior where $\mathbb{E}[\sigma^Z_1 \sigma^Z_2] \neq \mathbb{E}[\sigma^Z_1] \mathbb{E}[\sigma^Z_2]$